In [17]:
# Human's emotion recognition using thermal images
import numpy as np
import glob
import random
from numpy.linalg import svd as svd
from configparser import ConfigParser

np.set_printoptions(threshold=np.inf)

# Load db config:
dbconfig = ConfigParser()
dbconfig.read('database.ini')

# Assign dbDir with local config:
dbDir = dbconfig['LOCAL']['location'] #Database directory
eDir = {
    "angry":"anger",
    "disgusted":"disgust",
    "fear":"fear",
    "happy":"happy",
    "neutral":"neutral",
    "sad":"sadness",
    "surprised":"surprise"
} #Name of directory of each Emotion

In [18]:
def getSVXFiles(fd):
    return glob.glob(fd + '/*.svx')

In [19]:
def readSVX(file, frame):
    f = open(file,'rb') #Open file to read
    
    f.seek(128) #Skip 128bits
    [CA,CB] = np.divide(np.frombuffer(f.read(8), dtype=np.dtype('<i4'), count=2), 2**16)
    #Array from buffer
    #Data type: int32 (4 bytes/each - 'i4') 
    #Byte-order:  Litte-endian ('<')
    
    f.seek(1104) #Move to first frame to read
    
    img3d = np.zeros((frame,240,320))
    
    for i in range (0,frame):
        img = np.frombuffer(f.read(153600),dtype=np.dtype('>i2'))
        #Array from buffer
        #Data type: int16 (2 bytes/each - 'i2') 
        #Byte-order:  Big-endian ('>')
        img = img.dot(CB) + CA

        img2d = img.reshape(240,320) #Reshape 1 Dimension to 2 Dimensions for each frame
        img3d[i] = img2d #Add frame to array of frames
        
        f.seek(2592,1) #Move to next frame to read


    f.close()
    return img3d


#readSVX(dbDir+eDir["angry"]+"/anan1.svx",2)

In [20]:
###
#* Parameter: 
#*
#* -  fd: folder
def loadThermalGlasses(fd, frame):
    files = getSVXFiles(fd) #Get list SVX files in folder
    faces = []
    
    for file in files:
        data = readSVX(file,frame)
        
        for i in range(frame-21, frame, 8):
            faces += [data[i]]
            
    print('Total faces: ', len(faces))
    return faces

In [21]:
faces = loadThermalGlasses(dbDir + eDir["angry"],300)

Total faces:  42


In [22]:
def divideDataset(faces):
    #Divide Dataset into 2, 70% for training & 30% for testing
    train = []
    test = []
    dsLength = len(faces)
    limitTest = dsLength*0.3
    
    for face in faces:
        if len(test) < limitTest and random.randint(1,10) >= 7:
            test += [face]
        else: 
            train += [face]
    
    return [train,test]

In [23]:
[train,test] = divideDataset(faces)

print(len(train),len(test))

29 13


In [24]:
def meanface(faces):
    total = np.zeros(faces[0].shape)
    count = len(faces)
    
    total = sum(faces)
    
    return total / count

In [25]:
#meanface(train)

In [26]:
# TODO: Read about SVD function
def training(faces, trainedFiles, k):
    avg = meanface(faces)
    
    M = []
    for face in faces:
        W = face - avg
#         print(W.shape)
        M += [W.reshape(-1)] # Convert face to 1 dimension 
    M = np.array(M).transpose()
    print(M.shape)
    [U,W,V] = svd(M, full_matrices = False)
    print(U.shape)
    print(W.shape)
    print(V.shape)
#     print(M[:,0])
    trainweights = []
#     print(U.transpose())
    Ut = U.transpose()
    for i in range(0, M.shape[1]):
        trainweights += [Ut * M[:,i]]
    trainweights = np.array(trainweights)
    print(trainweights.shape)

In [27]:
training(train,'abc',15)

(76800, 29)
(76800, 29)
(29,)
(29, 29)
(29, 29, 76800)


In [29]:
test[0].shape

(240, 320)

In [ ]:
print(M[::][0])